## Read data from database

In [ ]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [ ]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [ ]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [ ]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
table_suffix = ["0","3","9","A","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where (CNT_CODE = 1 or MEM_EXECUTE = 1) and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201704')
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

In [ ]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
table_suffix = ["0","3","9","A","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where (CNT_CODE = 1 or MEM_EXECUTE = 1) and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201705')
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

## Check and split data

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 10000

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines >= 4 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines >= 4 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

In [ ]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [ ]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

## Autoencoder

In [ ]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, batch_size=32, dim=10000, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, X = self.__data_generation(list_IDs_temp)

        return X, X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > self.dim:
                bytes_data = [int(single_byte) for single_byte in in_file.read(self.dim)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data

        X = X.reshape((-1, 100, 100, 1)) / 255.0
        return X, X

In [ ]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Conv2D, Multiply, GlobalMaxPooling1D, Dropout, Activation, MaxPooling2D
from keras.layers import UpSampling2D
from keras.models import load_model, Model
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split

class Autoencoder():
    def __init__(self, autoencoder_name, encoder_name, input_shape, conv_filter_num, conv_kernel_size, pooling_size,
                 dense_num):
        self.autoencoder_name = autoencoder_name
        self.encoder_name = encoder_name
        self.input_shape = input_shape
        self.conv_filter_num = conv_filter_num
        self.conv_kernel_size = conv_kernel_size
        self.pooling_size = pooling_size
        self.dense_num = dense_num
        self.autoencoder = None
        self.encoder = None

    def get_model(self):
        input_img = Input(self.input_shape)

        x = Conv2D(self.conv_filter_num, self.conv_kernel_size, activation='relu', padding='same')(input_img)
        x = MaxPooling2D(self.pooling_size, padding='same')(x)
        encoded = Dense(self.dense_num, activation='sigmoid')(x)

        x = Conv2D(self.conv_filter_num, self.conv_kernel_size, activation='relu', padding='same')(encoded)
        x = UpSampling2D(self.pooling_size)(x)
        decoded = Conv2D(1, self.conv_kernel_size, activation='sigmoid', padding='same')(x)

        self.autoencoder = Model(inputs=input_img, outputs=decoded)
        self.encoder = Model(inputs=input_img, outputs=encoded)

        self.autoencoder.compile(loss='mean_squared_error', optimizer=RMSprop())
        self.autoencoder.summary()

    def train(self, train_df, max_epoch, batch_size=32):
        self.get_model()
        partition_train, partition_validation = train_test_split(range(len(train_df)), test_size=0.05,
                                                                 random_state=1234)
        print('Length of the train: ', len(partition_train))
        print('Length of the validation: ', len(partition_validation))

        #         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/autoencoder/models/{epoch:04d}-{val_loss:.5f}_12_3_a_0.h5"
        #         early_stopping = EarlyStopping("val_loss", patience=2, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point]

        # Generators
        training_generator = DataGenerator(partition_train, train_df, batch_size)
        validation_generator = DataGenerator(partition_validation, train_df, batch_size)

        self.autoencoder.fit_generator(generator=training_generator,
                                       validation_data=validation_generator,
                                       use_multiprocessing=True,
                                       epochs=max_epoch,
                                       workers=6,
                                       callbacks=callbacks_list)
        self.autoencoder.save(self.autoencoder_name)
        self.encoder.save(self.encoder_name)


In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [ ]:
autoencoder = Autoencoder('autoencoder.h5', 'encoder.h5', (100, 100, 1,), 32, 4, 25, 1)
autoencoder.train(x_train, max_epoch=3, batch_size=16)

In [ ]:
from keras.models import Model

model_dir = '/home/zhaoqi/autoencoder/models/'
f_name = '0002-0.08092_12_3_a_0.h5'
c_model = load_model(model_dir + f_name)

model_f = Model(c_model.input, c_model.layers[-4].output)

train_generator = DataGenerator(range(len(x_train)), x_train, 32, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, 32, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, 32, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

In [31]:
malcon_train_x = malcon_train_x.reshape(len(malcon_train_x), 16)
malcon_val_x = malcon_val_x.reshape(len(malcon_val_x), 16)
malcon_test_x = malcon_test_x.reshape(len(malcon_test_x), 16)

In [34]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [35]:
def get_max_feature(origin_data, feature_data):
    feature_data = pd.DataFrame(feature_data)
    feature_data['mw_file_hash'] = origin_data.iloc[0:len(feature_data)][['mw_file_hash']]
    feature_data = feature_data.groupby('mw_file_hash').max().merge(origin_data[['mw_file_hash','mw_em_f','mw_num_engines']], how='inner', on='mw_file_hash')
    tmp_ember = get_ember_feature(feature_data[['mw_em_f']])
    tmp_label = feature_data[['mw_num_engines']].mw_num_engines
    tmp_data = feature_data.drop(['mw_num_engines', 'mw_em_f', 'mw_file_hash'], axis=1)
    return tmp_data, tmp_label, tmp_ember

In [36]:
max_train_data, max_train_label, max_train_ember = get_max_feature(x_train, malcon_train_x)
max_val_data, max_val_label, max_val_ember = get_max_feature(x_val, malcon_val_x)
max_test_data, max_test_label, max_test_ember = get_max_feature(x_test, malcon_test_x)

In [39]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 16+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:16] = m_data.iloc[index]
        m_x[index, 16:128+2351] = e_data[index]  
    return m_x

In [40]:
merge_train_x = merge_feature(max_train_data, max_train_ember)
merge_val_x = merge_feature(max_val_data, max_val_ember)
merge_test_x = merge_feature(max_test_data, max_test_ember)

In [41]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, valid_sets=valid_sets, early_stopping_rounds=10)
    y_pred = model.predict(x_val, num_iteration=model.best_iteration)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [ ]:
model_m = get_model(merge_train_x[:,:], max_train_label, merge_val_x[:,:], max_val_label )

In [53]:
y_p = model_m.predict(merge_test_x[:,:])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

loss : 0.33634
auc score : 0.96708
accuracy score : 0.88979
thre: 0.9975493706
fp:  0.0000000000
recall:  0.0001534072


(0.9670753330017884, 0.3363407473644063, 0.00015340717331942442)